In [1]:
import json

def organize(input_rows):
    """
    Organize rows so parent items always come before their child items.
    """
    rows = input_rows.copy()
    parents = [None]
    results = []
    while len(rows):
        for i, row in enumerate(rows):
            if row['parent_id'] in parents:
                r = rows.pop(i)
                parents.append(r['id'])
                results.append(r)
    return results

def organize_json(json_str):
    rows = json.loads(json_str)
    return json.dumps(organize(rows), indent=2)



In [8]:
import unittest
import random

class OrganizeTests(unittest.TestCase):
    
    def setUp(self):
        self.rows = [
            {
                "name": "Men > Accessories > Belts > Leather",
                "id": 68,
                "parent_id": 60
            },
            {
                "name": "Women > Clothing > Tops",
                "id": 78,
                "parent_id": 75
            },
            {
                "name": "Women > Clothing > Tops > Blouses > Short Sleeves",
                "id": 80,
                "parent_id": 79
            },
            {
                "name": "Men > Accessories > Ties",
                "id": 64,
                "parent_id": 1
            },
            {
                "name": "Women > Clothing",
                "id": 75,
                "parent_id": 25
            },
            {
                "name": "Men > Shirts > Tees",
                "id": 33,
                "parent_id": 3,
            },
            {
                "name": "Men > Accessories",
                "id": 1,
                "parent_id": 20,
            },
            {
                "name": "Men > Accessories > Watches",
                "id": 57,
                "parent_id": 1
            },
            {
                "name": "Men",
                "id": 20,
                "parent_id": None
            },
            {
                "name": "Women > Clothing > Tops > Blouses",
                "id": 79,
                "parent_id": 78
            },
            {
                "name": "Women",
                "id": 25,
                "parent_id": None
            },
            {
                "name": "Men > Shirts",
                "id": 3,
                "parent_id": 20,
            },
            {
                "name": "Men > Accessories > Belts",
                "id": 60,
                "parent_id": 1
            },
        ]

    def test_empty(self):
        """
        Passing an empty list should return an empty list.
        """
        self.assertEqual(organize_json("[]"), "[]")

    def test_deterministic(self):
        """
        In the example given in the instructions, there can be only one valid solution.
        This test verifies that the sole valid solution is produced.
        """
        rows = '[{"name":"Accessories","id":1,"parent_id":20},{"name":"Watches","id":57,"parent_id":1},{"name":"Men","id":20,"parent_id":null}]'

        output = organize_json(rows)

        expected_output = '[{"name":"Men","id":20,"parent_id":null},{"name":"Accessories","id":1,"parent_id":20},{"name":"Watches","id":57,"parent_id":1}]'

        self.assertEqual(json.loads(output), json.loads(expected_output))

    def test_variations(self):
        """
        Generates 10 variations by shuffling the test rows, then
        verifies that no integrity errors are produced.
        """
        for i in range(10):
            random.shuffle(self.rows)
            output = organize(self.rows)
            seen = []
            for r in output:
                seen.append(r['id'])
                if r['parent_id']:
                    # No matter how the items are shuffled,
                    # the rows in the output should be sorted
                    # in such a way that each child's parent
                    # precedes it.
                    self.assertTrue(r['parent_id'] in seen)

    def test_complex(self):
        """
        Generates an input consisting of 20,100 rows.
        The first 10,000 rows have no parent.
        The second 10,000 rows are children of those first 10,000 rows, 
        with a 1:1 mapping of parent to child.
        The final 100 rows illustrate deep nesting, where each row is a child
        of the preceding row.
        Then, the rows are shuffled, we run our `organize` function,
        and verify that no integrity errors are present.
        """

        rows = []
        i = 0
        while i < 10000:
            rows.append({"name": f"Row {i}", "id": i, "parent_id": None})
            i += 1
        while i < 20000:
            rows.append({"name": f"Row {i}", "id": i, "parent_id": i-10000})
            i += 1
        while i < 20100:
            rows.append({"name": f"Row {i}", "id": i, "parent_id": i-1})
            i += 1

        random.shuffle(rows)
        output = organize(rows)
        seen = []
        for r in output:
            seen.append(r['id'])
            if r['parent_id']:
                # No matter how the items are shuffled,
                # the rows in the output should be sorted
                # in such a way that each child's parent
                # precedes it.
                self.assertTrue(r['parent_id'] in seen)


    @unittest.expectedFailure
    def test_unorganized(self):
        """
        This is essentially a test of a test, where we verify that
        if we don't first send the test rows through the `organize`
        function, it leads to integrity errors.
        """
        seen = []
        for r in self.rows:
            seen.append(r['id'])
            if r['parent_id']:
                # Unlike in `test_variations`, where we organize the
                # rows first, this test evaluates the rows without
                # organizing them, so we should expect it to fail.
                self.assertTrue(r['parent_id'] in seen)                          


In [9]:
unittest.main(argv=[''], verbosity=3, exit=False)

test_complex (__main__.OrganizeTests) ... ok
test_deterministic (__main__.OrganizeTests) ... ok
test_empty (__main__.OrganizeTests) ... ok
test_unorganized (__main__.OrganizeTests) ... expected failure
test_variations (__main__.OrganizeTests) ... ok

----------------------------------------------------------------------
Ran 5 tests in 2.112s

OK (expected failures=1)
